In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [27]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
StopWords=ENGLISH_STOP_WORDS

In [28]:
train=pd.read_csv('Train.csv', delimiter=',')

In [29]:
train=train.fillna(method='ffill')

In [30]:
text_1=train['safe_text'][0]

In [31]:
train

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.000000
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.000000
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.000000
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.000000
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.000000
...,...,...,...,...
9996,IU0TIJDI,Living in a time where the sperm I used to was...,1.0,1.000000
9997,WKKPCJY6,<user> <user> In spite of all measles outbrea...,1.0,0.666667
9998,ST3A265H,Interesting trends in child immunization in Ok...,0.0,1.000000
9999,6Z27IJGD,CDC Says Measles Are At Highest Levels In Deca...,0.0,1.000000


In [32]:
import re

In [33]:
review=re.sub('[^a-zA-z0-9_]',' ',train['safe_text'][0])

In [34]:
review

'Me  amp  The Big Homie meanboy3000  MEANBOY  MB  MBS  MMR  STEGMANLIFE   Stegman St   url '

In [35]:
sub=re.sub('[^a-zA-Z0-9_]',' ', train['safe_text'][4])

In [36]:
sub

'Thanks to  user  Catch me performing at La Nuit NYC 1134 1st ave  Show starts at 6   jennifair  mmr   url '

In [37]:
corpus=[]
for i in range(0,10001):
    review =re.sub('[^a-zA-Z0-9_]',' ',train['safe_text'][i])
    review=review.lower()
    review=review.split('url')
    review=' '.join(review)
    review=review.split('user')
    review=' '.join(review)
    review=review.split('br')
    review=' '.join(review)
    review=review.split()
    review=' '.join(review)
    review=[ps.stem(word) for word in review.split() if word not in set(StopWords)]
    review=' '.join(review)
    corpus.append(review)

In [39]:
corpus[0]

'amp big homi meanboy3000 meanboy mb mb mmr stegmanlif stegman st'

In [40]:
train['label']=np.round(train.label, decimals=0)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

In [42]:
cv=CountVectorizer(max_features=5000)

In [43]:
X=cv.fit_transform(corpus).toarray()

In [44]:
X.shape

(10001, 5000)

In [45]:
y=train['label'].values

In [46]:
from sklearn.model_selection import StratifiedKFold

In [47]:
skf=StratifiedKFold(n_splits=10, random_state=2, shuffle=True)
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [48]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

In [49]:
xgb=RandomForestRegressor(n_estimators=5,random_state=2).fit(X_train,y_train)

In [50]:
prediction=xgb.predict(X_test)

In [51]:
from sklearn.metrics import mean_squared_error

In [52]:
print(np.sqrt(mean_squared_error(y_test,prediction)))

0.649891498225844


In [99]:
test=pd.read_csv('Test.csv', delimiter=',')

In [100]:
test=test.fillna(method='ffill')

In [101]:
corpus_test=[]
for i in range(0,5177):
    review =re.sub('[^a-zA-Z0-9_]',' ',test['safe_text'][i])
    review=review.lower()
    review=review.split('url')
    review=' '.join(review)
    review=review.split('user')
    review=' '.join(review)
    review=review.split('br')
    review=' '.join(review)
    review=review.split()
    review=' '.join(review)
    review=[ps.stem(word) for word in review.split() if word not in set(StopWords)]
    review=' '.join(review)
    corpus_test.append(review)

In [102]:
test.shape

(5177, 2)

In [103]:
test_corpus=cv.transform(corpus_test).toarray()

In [104]:
test_pred=xgb.predict(test_corpus)

In [105]:
(pd.Series(test_pred)==0.293054).sum()

0

In [110]:
sub=pd.read_csv('SampleSubmission.csv', delimiter=',')
new=pd.read_csv('latest_sub.csv', delimiter=',')

In [111]:
new_sub=sub.copy()

In [115]:
label=new['label'].values

array([0.29305428, 0.50866294, 0.10418266, ..., 0.24659401, 0.6886089 ,
       0.39452696])

In [133]:
new_sub.label=0.3*(test_pred) +0.7*(label)

In [134]:
new_sub.to_csv('latest_sub_7.csv', index=False)

In [26]:
rfc=RandomForestRegressor().fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [27]:
prediction_rfc=rfc.predict(X_test)

In [28]:
print(np.sqrt(mean_squared_error(y_test,prediction_rfc)))

0.625885228205666


In [28]:
from sklearn.model_selection import cross_val_score

In [30]:
accuracies=cross_val_score(estimator=xgb,X=X_train, y=y_train, cv=10)

In [31]:
accuracies.mean()

0.010157498253315245

In [32]:
accuracies.std()

0.03626327369651533

In [33]:
accuracies

array([ 0.00315594,  0.00336724, -0.01358039, -0.0116686 ,  0.00601731,
        0.0300426 ,  0.07060966,  0.00362649,  0.06766188, -0.05765714])